In [ ]:
import investpy
from datetime import datetime
import requests
import pandas as pd
import ta
import numpy as np
from time import sleep
from IPython.display import clear_output

In [ ]:
url = "https://price.tpbs.com.vn/api/StockBoardApi/getStockList"
payload = "{\"boardName\":\"VN30\"}"
payload2 = "{\"boardName\":\"HNX30\"}"
headers = {
    'Content-Type': 'text/plain'
}

def crawlData(url, payload, headers):
    response = requests.request("POST", url, headers=headers, data=payload).json()
    return eval(response['content'])


list_stock = []
list_stock.extend(crawlData(url, payload, headers))
# list_stock.extend(crawlData(url, payload2, headers))
print(list_stock)

['ACB', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB', 'HPG', 'KDH', 'MBB', 'MSN', 'MWG', 'NVL', 'PDR', 'PLX', 'PNJ', 'POW', 'SAB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIC', 'VJC', 'VNM', 'VPB', 'VRE']


In [ ]:
len(list_stock)

30

In [ ]:
lst_df = []
today = datetime.today().strftime("%d/%m/%Y")
for i in list_stock:
    try:
        df = investpy.get_stock_historical_data(stock=i, country='VietNam', from_date='01/01/2000', to_date=today)
        df['Symbol'] = i
        df['Return'] = df['Close'].pct_change(fill_method='ffill')
        df.drop('Currency', axis=1, inplace=True)
        lst_df.append(df)
        print(list_stock.index(i), " = ", i)
        clear_output(wait = True)
    except:
        search_result = investpy.search_quotes(text=i, countries=['VietNam'], products=['stocks'], n_results=1)
        historical_data = search_result.retrieve_historical_data(from_date='01/01/2000', to_date=today)
        historical_data['Symbol'] = i
        historical_data['Return'] = historical_data['Close'].pct_change(fill_method='ffill')
        historical_data.drop('Change Pct', axis=1, inplace=True)
        lst_df.append(historical_data)
        print("except: ", list_stock.index(i), " = ", i)
        clear_output(wait = True)

dt = pd.concat(lst_df)
dt.to_csv("VN30_raw.csv", )
print(dt)

               Open     High      Low    Close    Volume Symbol    Return
Date                                                                     
2006-11-22  20772.0  20772.0  18857.0  19582.0    267581    ACB       NaN
2006-11-23  21033.0  21033.0  18857.0  20743.0    300224    ACB  0.059289
2006-11-24  20308.0  22048.0  20308.0  21758.0    273165    ACB  0.048932
2006-11-27  21294.0  23412.0  19582.0  20598.0    182110    ACB -0.053314
2006-11-28  21758.0  21758.0  20308.0  20598.0     85042    ACB  0.000000
...             ...      ...      ...      ...       ...    ...       ...
2022-03-07  33300.0  33300.0  32800.0  32900.0  10892800    VRE -0.025185
2022-03-08  32750.0  33000.0  32500.0  32500.0   4935000    VRE -0.012158
2022-03-09  32500.0  32600.0  31600.0  31600.0   6560500    VRE -0.027692
2022-03-10  32100.0  32600.0  32000.0  32000.0  14045300    VRE  0.012658
2022-03-11  31700.0  32500.0  31500.0  31850.0   3607800    VRE -0.004687

[59013 rows x 7 columns]


In [ ]:
for stock in lst_df:
    stock['SMA_5'] = ta.trend.sma_indicator(stock['Close'], window=5)
    stock['SMA_15'] = ta.trend.sma_indicator(stock['Close'], window=15)
    stock['SMA_ratio'] = stock['SMA_15'] / stock['SMA_5']

print("Done")

Done


In [ ]:
for stock in lst_df:
    stock['SMA5_Volume'] = ta.trend.sma_indicator(stock['Volume'], window=5)
    stock['SMA15_Volume'] = ta.trend.sma_indicator(stock['Volume'], window=15)
    stock['SMA_Volume_Ratio'] = stock['SMA15_Volume'] / stock['SMA5_Volume']

print("Done")

Done


In [ ]:
for stock in lst_df:
    stock['ATR_5'] = ta.volatility.average_true_range(high=stock['High'], low=stock['Low'], close=stock['Close'], window=5)
    stock['ATR_15'] = ta.volatility.average_true_range(high=stock['High'], low=stock['Low'], close=stock['Close'], window=15)
    stock['ATR_Ratio'] = stock['ATR_5'] / stock['ATR_15']

print("Done")

Done


In [ ]:
for stock in lst_df:
    stock['ADX_5'] = ta.trend.adx(stock['High'], stock['Low'], stock['Close'], window=5)
    stock['ADX_15'] = ta.trend.adx(stock['High'], stock['Low'], stock['Close'], window=15)
    stock['ADX_15'] = stock['ADX_15'].replace(0, np.nan)

print("Done")

C:\Users\Admin\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in true_divide
  directional_index = 100 * np.abs((dip - din) / (dip + din))


Done


In [ ]:
for stock in lst_df:

    stock['Stochastic_5'] = ta.momentum.stoch(stock['High'], stock['Low'], stock['Close'], window=5, smooth_window=3)
    stock['Stochastic_15'] = ta.momentum.stoch(stock['High'], stock['Low'], stock['Close'], window=15, smooth_window=3)

    stock['Stochastic_%D_5'] = stock['Stochastic_5'].rolling(window = 5).mean()
    stock['Stochastic_%D_15'] = stock['Stochastic_5'].rolling(window = 15).mean()

    stock['Stochastic_Ratio'] = stock['Stochastic_%D_5']/stock['Stochastic_%D_15']

print("Done")

Done


In [ ]:
for stock in lst_df:
    stock['RSI_5'] = ta.momentum.rsi(stock['Close'], window=5)
    stock['RSI_15'] = ta.momentum.rsi(stock['Close'], window=15)
    stock['RSI_ratio'] = stock['RSI_5'] / stock['RSI_15']

print("Done")

Done


In [ ]:
for stock in lst_df:
    stock['MACD'] = ta.trend.macd(stock['Close'], window_slow=15, window_fast=5)

print("Done")

Done


In [ ]:
final_data = pd.concat(lst_df)
x = final_data.dropna()
x

,Open,High,Low,Close,Volume,Symbol,Return,SMA_5,SMA_15,SMA_ratio,...,ADX_15,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,RSI_5,RSI_15,RSI_ratio,MACD
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,21758.0,21903.0,21541.0,21686.0,183398,ACB,0.010155,21607.2,20971.000000,0.970556,...,25.093592,45.034483,72.526715,68.308864,59.549210,1.147099,68.712603,62.903213,1.092354,515.231459
2007-01-04,22338.0,22483.0,21758.0,22483.0,342745,ACB,0.036752,21830.6,21116.000000,0.967266,...,25.703732,100.000000,100.000000,69.677010,63.185447,1.102738,81.871143,68.653723,1.192523,655.267330
2007-01-05,22628.0,23209.0,22483.0,22846.0,346181,ACB,0.016146,22048.2,21284.266667,0.965352,...,26.701074,80.752916,89.991729,70.825870,67.090601,1.055675,85.373348,70.858164,1.204848,785.277116
2007-01-08,22773.0,23209.0,22048.0,22701.0,228067,ACB,-0.006347,22236.8,21414.800000,0.963034,...,27.078960,73.064687,85.993934,70.437084,69.642131,1.011415,77.862786,68.787792,1.131927,798.187944
2007-01-09,22773.0,22919.0,22338.0,22628.0,210028,ACB,-0.003216,22468.8,21530.800000,0.958253,...,27.431653,65.167866,83.981252,72.803990,71.377526,1.019985,73.778245,67.720467,1.089453,757.253097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07,33300.0,33300.0,32800.0,32900.0,10892800,VRE,-0.025185,33130.0,33800.000000,1.020223,...,11.306198,33.333333,17.741935,28.584410,41.391436,0.690588,38.473278,45.965404,0.837005,-409.288699
2022-03-08,32750.0,33000.0,32500.0,32500.0,4935000,VRE,-0.012158,32930.0,33700.000000,1.023383,...,11.225675,9.090909,4.838710,27.359114,39.830830,0.686883,32.880690,43.996226,0.747353,-510.119764
2022-03-09,32500.0,32600.0,31600.0,31600.0,6560500,VRE,-0.027692,32690.0,33520.000000,1.025390,...,11.807059,0.000000,0.000000,25.619984,37.164163,0.689373,23.338952,39.877802,0.585262,-735.182895


In [ ]:
len(set(x['Symbol']))

29

In [ ]:
print(set(x['Symbol']))

{'BVH', 'SSI', 'CTG', 'SAB', 'MWG', 'PDR', 'VNM', 'VJC', 'POW', 'ACB', 'GVR', 'VCB', 'VRE', 'FPT', 'VHM', 'HPG', 'PNJ', 'BID', 'VPB', 'VIC', 'GAS', 'NVL', 'TCB', 'MSN', 'HDB', 'MBB', 'TPB', 'KDH', 'PLX'}


In [ ]:
print(set(dt['Symbol']))

{'BVH', 'SSI', 'CTG', 'SAB', 'MWG', 'PDR', 'VNM', 'VJC', 'POW', 'ACB', 'GVR', 'VCB', 'VRE', 'FPT', 'VHM', 'HPG', 'PNJ', 'BID', 'VPB', 'VIC', 'GAS', 'NVL', 'TCB', 'MSN', 'HDB', 'MBB', 'TPB', 'STB', 'KDH', 'PLX'}


In [ ]:
t = dt[dt['Symbol'] == 'STB']
t

,Open,High,Low,Close,Volume,Symbol,Return,SMA_5,SMA_15,SMA_ratio,...,ADX_15,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,RSI_5,RSI_15,RSI_ratio,MACD
Date,,,,,,,,,,,,,,,,,,,,,
2013-07-30,16800.0,17100.0,16800.0,17100.0,421830,STB,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-07-31,16800.0,17000.0,16800.0,17000.0,259220,STB,-0.005848,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-08-01,17000.0,17000.0,16800.0,17000.0,247020,STB,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-08-02,16800.0,17000.0,16800.0,17000.0,143610,STB,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-08-05,16900.0,17000.0,16800.0,17000.0,103740,STB,0.000000,17020.0,NaN,NaN,...,NaN,66.666667,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07,31200.0,32000.0,31200.0,31500.0,16727500,STB,-0.015625,31850.0,32856.666667,1.031606,...,21.180367,21.052632,12.698413,26.153488,36.586579,0.714838,29.078392,41.145333,0.706724,-777.743579
2022-03-08,31050.0,31650.0,30650.0,30850.0,16477900,STB,-0.020635,31490.0,32680.000000,1.037790,...,22.006914,10.526316,5.555556,21.592085,35.955000,0.600531,21.697300,38.008206,0.570858,-904.793705
2022-03-09,31000.0,31300.0,30500.0,31100.0,14911000,STB,0.008104,31460.0,32513.333333,1.033482,...,22.873504,29.268293,16.000000,26.291897,35.642068,0.737665,30.213718,39.896633,0.757300,-889.123207


In [ ]:
x = ta.trend.adx(stock['High'], stock['Low'], stock['Close'], window=15)
x

Date
2017-11-10     0.000000
2017-11-13     0.000000
2017-11-14     0.000000
2017-11-15     0.000000
2017-11-16     0.000000
                ...    
2022-03-07    11.306198
2022-03-08    11.225675
2022-03-09    11.807059
2022-03-10    12.349685
2022-03-11    13.204633
Name: adx, Length: 1080, dtype: float64